#1유형

##1-1

20L가격과 5L가격이 모두 0원이 아닌 데이터만 필터를 한 후, 각 row별로 20L가격과 5L가격의 차이를 ‘차이가격’ 이라 부른다고 하자. 시도명 별 차이가격의 평균가격을 비교할때 그 값이 가장 큰 금액을 반올림하여 소숫점 이하 1자리까지 구하여라

In [98]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p1_1_.csv')
df.head(5)

pd.set_option('display.max_columns', None)

result=df.loc[df['5L가격']!=0 & df['20L가격']]
# r=result.copy()
result['차이가격']=abs(result['5L가격']-result['20L가격'])
result = result[['시도명','차이가격']].groupby(by=['시도명']).mean().round(1)
result.max()
# help(result.groupby)

<ipython-input-98-00fc61870a90>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['차이가격']=abs(result['5L가격']-result['20L가격'])


차이가격    593.0
dtype: float64

##1-2

BMI는 몸무게(kg) / (키(M) * 키(M)) 로 정의 된다. 초고도 비만은 BMI 25이상 , 고도 비반은 BMI 25미만 - 23이상 , 정상은 23미만 - 18.5이상 저체중은 18.5미만으로 정의 된다. 주어진 데이터에서 초고도비만 인원 + 저체중 인원 의 숫자는?


In [52]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p1_2_.csv')
# df.head(5)

,ID,height(cm),weight(kg)
0,user_1,173.0,67.5
1,user_2,149.5,82.5
2,user_3,143.8,64.6
3,user_4,163.1,87.2
4,user_5,159.6,77.1


In [78]:
import math
r=df
r['bmi']=(r['weight(kg)']/(pow(r['height(cm)']/100 , 2))).round(2)

def check(x):
    if x>=25:
        return '초고도'
    if x<25 and x>=23:
        return '고도'
    if x<23 and x>=18.5:
        return '정상'
    if x<18.5:
        return '저체중'

r['check'] = r['bmi'].apply(check)

a= len(r.loc[r.check=='초고도'])
b= len(r.loc[r.check=='저체중'])
print(a+b)
# r.head()
# help(df.value_counts)

8998


##1-3

순유입인원은 초중고 도내,도외 전입인원에서 초중고 도내, 도외 전출인원을 뺀값이다. 각년도별로 가장 큰 순유입인원을 가진 지역구의 순유입인원을 구하고 전체 기간의 해당 순유입인원들의 합을 구하여라

In [96]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p1_3.csv')

r=df
r['전입']=r['초등학교_전입_도내']+r['초등학교_전입_도외']+r['중학교_전입_도내']+r['중학교_전입_도외']+r['고등학교_전입_도내']+r['고등학교_전입_도외']
r['전출']=r['초등학교_전출_도내']+r['초등학교_전출_도외']+r['중학교_전출_도내']+r['중학교_전출_도외']+r['고등학교_전출_도내']+r['고등학교_전출_도외']
r['순유입인원']=r['전입']-r['전출']
r2=r[['년도','순유입인원','지역']].sort_values(['년도','순유입인원']).drop_duplicates('년도',keep='last')

print(r2['순유입인원'].sum())
r2.head(20)

13853


,년도,순유입인원,지역
22,2012,949,강남구
47,2013,1339,강남구
72,2014,2225,강남구
97,2015,1498,강남구
122,2016,774,강남구
149,2017,932,강동구
172,2018,1158,강남구
197,2019,1821,강남구
222,2020,1762,강남구
247,2021,1395,강남구


#2유형

예측 변수 price, test.csv에 대해 ID별로 price 값을 예측하여 제출, 제출 데이터 컬럼은 ID와 price 두개만 존재해야함. 평가지표는 rmse

In [173]:
from os import DirEntry
import pandas as pd
import math
from sklearn.model_selection import train_test_split


import sklearn.linear_model,sklearn.ensemble, xgboost, sklearn.metrics

train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv')

y_train=train.pop('price')
x_train=train
x_test = test
# print(train.shape)
drop_col = ['ID']
x_train_drop = x_train.drop(columns=drop_col)
x_test_drop =  x_test.drop(columns=drop_col)

x_train_dum = pd.get_dummies(x_train_drop)
x_test_dum = pd.get_dummies(x_test_drop)
x_test_dum = x_test_dum.reindex(columns=x_train_dum.columns, fill_value=0)


y=y_train

X_train, X_val, Y_train, Y_val = train_test_split(x_train_dum,y, test_size=0.33, random_state=42)

print(X_train.shape,x_test_dum.shape)

lr = sklearn.linear_model.LinearRegression()
rf = sklearn.ensemble.RandomForestRegressor()
xgb = xgboost.XGBRegressor()

lr.fit(X_train, Y_train)
lr_pred=lr.predict(X_val)
lr_mse = sklearn.metrics.mean_squared_error(Y_val,lr_pred)
lr_rmse = pow(lr_mse,0.5)
print('lr', lr_rmse)

rf.fit(X_train, Y_train)
rf_pred=rf.predict(X_val)
rf_mse = sklearn.metrics.mean_squared_error(Y_val,rf_pred)
rf_rmse = pow(rf_mse,0.5)
print('rf', rf_rmse)

xgb.fit(X_train, Y_train)
xgb_pred=xgb.predict(X_val)
xgb_mse = sklearn.metrics.mean_squared_error(Y_val,xgb_pred)
xgb_rmse = pow(xgb_mse,0.5)
print('xgb', xgb_rmse)

pred = xgb.predict(x_test_dum)
result = pd.DataFrame({'ID':x_test.ID, 'price':pred}).to_csv('00001234',index=False)

r2 = pd.read_csv('00001234')
r2.head()

# print(X_val.shape)
# display(y_train.head())
# display(x_test.head())
# display(x_train.nunique())
# test.head()
# help()

(6581, 39) (3296, 39)
lr 5627.205206932107
rf 2546.9544193944985
xgb 2365.3848327202445


,ID,price
0,0,4053.1558
1,2,57795.2600
2,14,17008.2070
3,15,14084.4360
4,18,15842.9080


#3유형

이 학생들의 키의 95% 신뢰구간을 구하고자 한다.

In [174]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p3_1.csv')
df.head()

,ID,height
0,user_1,164.57
1,user_2,174.99
2,user_3,171.41
3,user_4,162.47
4,user_5,167.11


##3-1

a.55명 학생들의 키에 대한 표본 평균을 구하여라(반올림하여 소숫점 3째자리까지

In [177]:
r = df
result = round(r.height.mean(),3)
print(result)

169.937


b.t분포 양쪽 꼬리에서의 t 값을 구하여라 (반올림하여 소수4째자리까지)

In [200]:
import numpy as np
from scipy.stats import t

std = np.std(df.height, ddof=1)
n = len(df.height)

# 신뢰수준, 자유도
confidence_level = 0.95
ddof = n - 1

# t 분포의 양쪽 꼬리에서의 t값
t_value = round(t.ppf((1 + confidence_level) / 2, ddof),4)
print(t_value)

2.0049


##3-2

A,B,C 세 공장에서 생산한 동일한 제품의 길이 데이터 이다. DataUrl = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p3_2.csv 공장간의 제품 길이 차이가 유의미한지 확인 하려한다.

In [205]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p3_2.csv')
df.nunique()

ID        3
value    30
dtype: int64

a.3 그룹의 데이터에 대해 크루스칼-왈리스 검정을 사용하여 검정 통계량을 반올림하여 소숫점 이하 3자리까지 구하여라

In [225]:
from scipy import stats
r=df
ra = r.loc[r.ID=='A'].value.values
rb = r.loc[r.ID=='B'].value.values
rc = r.loc[r.ID=='C'].value.values
round(stats.kruskal(ra,rb,rc).statistic, 3)
# print(type(ra))

# help(stats.kruskal)

6.521

3 그룹의 데이터에 대해 크루스칼-왈리스 검정을 사용하여 p-value를 반올림하여 소숫점 이하 3자리까지 구하여라. 귀무가설과 대립가설중 0.05 유의수준에서 유의한 가설을 출력하라

In [227]:
if stats.kruskal(ra,rb,rc).pvalue<0.05:
    print("귀무가설 기각, 대립가설 채택")

else :
    print('귀무가설 채택')


귀무가설 기각, 대립가설 채택
